In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timezone
from tqdm import tqdm
import matplotlib.pyplot as plt
import plotly.io as pio
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
import warnings
warnings.filterwarnings("ignore")
import plotly.express as px


In [2]:
data_anly_1= pd.read_csv('ch5_data/analyse_all_dates/abs.csv') 

data_anly_2= pd.read_csv('ch5_data/analyse_all_dates/range_width_1.csv') 

data_anly_3= pd.read_csv('ch5_data/analyse_all_dates/range_width_30.csv') 

data_anly_4= pd.read_csv('ch5_data/analyse_all_dates/range_with_halfE.csv') 


In [3]:
data_anly_1['Datetime'] = pd.to_datetime(data_anly_1['Datetime'])
data_anly_1 = data_anly_1[:len(data_anly_1)-10]
data_anly_1

,Unnamed: 0,Datetime,P_W_list,p_bss_values,E,P_W_shift,original_flucuation,P_CW,P_CW_shift,after_flucuation
0,0,2022-01-01 00:00:00,1803.48,0.00,338.16,1803.48,0.00,1803.48,1803.48,0.00
1,1,2022-01-01 00:15:00,1962.39,-466.09,446.76,1803.48,158.91,1496.30,1803.48,-307.18
2,2,2022-01-01 00:30:00,1801.63,-305.33,517.90,1962.39,-160.76,1496.30,1496.30,0.00
3,3,2022-01-01 00:45:00,1824.41,-328.11,594.35,1801.63,22.78,1496.30,1496.30,0.00
4,4,2022-01-01 01:00:00,1661.57,-165.26,632.86,1824.41,-162.84,1496.31,1496.30,0.01
...,...,...,...,...,...,...,...,...,...,...
34995,34995,2022-12-31 12:45:00,2184.38,-45.10,404.30,2182.35,2.03,2139.28,2138.07,1.21
34996,34996,2022-12-31 13:00:00,2178.86,-38.35,413.23,2184.38,-5.52,2140.51,2139.28,1.23
34997,34997,2022-12-31 13:15:00,2189.13,-47.71,424.35,2178.86,10.27,2141.42,2140.51,0.91
34998,34998,2022-12-31 13:30:00,2168.09,-25.79,430.36,2189.13,-21.04,2142.30,2141.42,0.88


- 仿真程序流程
- 		画图
- 		仿真结果和分析
- 	仿真的结果包含
- 		4个方案的全年运行结果
- 			只需要包含功率列和能量列
- 	仿真的结果分析包含
- 		4个方案的全年考核量
- 		4个方案的连续向上波动、连续向下波动、一上一下波动的优化统计（观察对于典型场景的总体应对效果）
- 			原先连续波动“长度-波动总值”分布
- 仿真的可视化
- 		每个方案验证了前面猜想的图（需要观察数据进行画图）
- 		日尺度图
- 		月尺度图（说明选择该图的原因（比如每种波动向上波动向下波动频繁上下波动长波动均包含））
- 4个方案的典型日图（可采取同一天，用两天）（体现不同方案的应对）
- 		4个方案的典型月图（可整体观察优化效果，而非局限于日）


# 可视化函数

In [4]:
def visualization_interactive(data_season_test, upper_baseline, lower_baseline,start,end):# ,route
    # 函数功能：可视化
    # 函数输入：数据集，上下准线
    data_season = data_season_test.copy()[start:end]

    # 使用Plotly创建交互式图表
    fig = make_subplots(rows=1, cols=1) # 实际上只画了一个图

    # 添加数据线
    fig.add_trace(go.Scatter(x=data_season['Datetime'], y=data_season['original_flucuation'], mode='lines+markers', name='波动值'))
    fig.add_trace(go.Scatter(x=data_season['Datetime'], y=data_season['after_flucuation'], mode='lines+markers', name='后波动值'))
    fig.add_trace(go.Scatter(x=data_season['Datetime'], y=data_season['p_bss_values'], mode='lines+markers', name='储能功率值'))
    fig.add_trace(go.Scatter(x=data_season['Datetime'], y=data_season['E'], mode='lines+markers', name='储能能量值'))

    # 添加基线
    fig.add_hline(y=upper_baseline, line_dash="dash", line_color="orange", annotation_text="上波动考核线", annotation_position="bottom right")
    fig.add_hline(y=lower_baseline, line_dash="dash", line_color="red", annotation_text="下波动考核线", annotation_position="top right")

    # 设置图表布局
    fig.update_layout(
        title='波动情况分析',
        xaxis_title='日期时间',
        yaxis_title='波动值 (MW)',
        legend_title='图例',
        font=dict(family="SimHei", size=12)
    )
    
    # 设置X轴为日期时间格式
    fig.update_xaxes(tickformat="%Y-%m-%d %H:%M:%S")
    # fig.savefig('ch5fig/range_test.png')
    # 显示图表
    # fig.write_image(route)
    fig.show()


In [5]:

# 分别绘制并输出4张图片，每张图片显示一个季节的波动情况和基线
data_season_test = data_anly_1.copy()
visual_length = 96

# 考核标准的衡量
upper_baseline = 50  # MW
lower_baseline = -50  # MW

# 使用修改后的函数绘制交互式图像
visualization_interactive(data_season_test, upper_baseline, lower_baseline,0,visual_length)#,fig_route


In [6]:
# 分别绘制并输出4张图片，每张图片显示一个季节的波动情况和基线
data_season_test = data_anly_2.copy()
visual_length = 96

# 考核标准的衡量
upper_baseline = 50  # MW
lower_baseline = -50  # MW

# 使用修改后的函数绘制交互式图像
visualization_interactive(data_season_test, upper_baseline, lower_baseline,0,visual_length)#,fig_route


# 考核电量统计定义

In [7]:
def refine(df,folder_name): 
    # 函数功能：统计考核量（需要写一段代码算出考核量
    # 函数输入：全年数据窗口
    # 函数输出：统计结果dataframe，考核量列表（包含减少的考核电量、原考核电量、后考核电量）
    df_result = df.copy()
    df_result['mean_original_flucuation'] = df_result['abs_original_flucuation'].rolling(window=10,min_periods=1).mean()#.shift(-window_size+1)
    df_result['original_refine_electricity'] = df_result['mean_original_flucuation'][df_result['mean_original_flucuation']>50] - 50
    original_refine_amount = df_result['original_refine_electricity'].sum()*10*15/60 # 单位MW.h
    df_result['mean_after_flucuation'] = df_result['abs_after_flucuation'].rolling(window=10,min_periods=1).mean()#.shift(-window_size+1)
    df_result['after_refine_electricity'] = df_result['mean_after_flucuation'][df_result['mean_after_flucuation']>50] - 50
    after_refine_amount = df_result['after_refine_electricity'].sum()*10*15/60 # 单位MW.h
    filename = 'refine.csv'
    file_path = os.path.join(folder_name, filename)
    reduced_refine_amount = original_refine_amount - after_refine_amount
    df_result.to_csv(file_path,index=False)
    refine_list = [reduced_refine_amount,original_refine_amount,after_refine_amount]
    return df_result,refine_list

# 考核点统计定义

In [8]:
def refine_num(df): # 统计考核点的数量（）# 没用的指标！
    # 输入 目标分析数据集
    # 输出 列表包含
    # 原受考核点数量、原向上波动考核点数量、原向下波动考核点数量、优化后受考核点数量、优化后向上波动考核点数量、优化后向下波动考核点数量、
    # 原首考核点比例、原向上波动考核点波动、原向下波动考核点比例、优化后同理

    df_result = df.copy()
    original_refine_points = df_result['abs_original_flucuation'][df_result['abs_original_flucuation'] > upper_baseline].count()
    original_upper_refine_points = df_result['original_flucuation'][df_result['original_flucuation'] > upper_baseline].count()
    original_lower_refine_points = df_result['original_flucuation'][df_result['original_flucuation'] < lower_baseline].count()
    
    after_refine_points = df_result['abs_after_flucuation'][df_result['abs_after_flucuation'] > upper_baseline].count()
    after_upper_refine_points = df_result['after_flucuation'][df_result['after_flucuation'] > upper_baseline].count()
    after_lower_refine_points = df_result['after_flucuation'][df_result['after_flucuation'] < lower_baseline].count()

    original_abs_ratio = original_refine_points/len(df_result)
    original_upper_ratio = original_upper_refine_points/len(df_result)
    original_lower_ratio = original_lower_refine_points/len(df_result)

    after_abs_ratio = after_refine_points/len(df_result)
    after_upper_ratio = after_upper_refine_points/len(df_result)
    after_lower_ratio = after_lower_refine_points/len(df_result)

    refine_num_list = [original_refine_points,
                       original_upper_refine_points,
                       original_lower_refine_points,
                       after_refine_points,
                       after_upper_refine_points,
                       after_lower_refine_points,
                       original_abs_ratio,
                       original_upper_ratio,
                       original_lower_ratio,
                       after_abs_ratio,
                       after_upper_ratio,
                       after_lower_ratio]
    return refine_num_list


# 连续波动特点统计定义

In [9]:
def continuou_fluctuation_analysis(df,mode,time_mode):
    # 功能描述：分析连续波动的各项指标
    # 函数输入：原始数据集，模式，保存文件夹
    # 函数输出：分析数据集
    df_positive_copy = df.copy()

    if time_mode == 'original':
        df_positive_copy['Fluctuation15M'] = df_positive_copy['original_flucuation']
    elif time_mode == 'after':
        df_positive_copy['Fluctuation15M'] = df_positive_copy['after_flucuation']


    if mode == 'positive':
        df_positive_copy['is_positive'] = df_positive_copy['Fluctuation15M'] > 0
    elif mode == 'negative':
        df_positive_copy['is_positive'] = df_positive_copy['Fluctuation15M'] < 0
        df_positive_copy['is_positive'] = -df_positive_copy['is_positive']
    
    # 进行绝对值的处理
    df_positive_copy['is_positive'] = abs(df_positive_copy['is_positive'])
    
    # 检测连续正数序列的开始
    df_positive_copy['new_group'] = df_positive_copy['is_positive'] & (df_positive_copy['is_positive'] != df_positive_copy['is_positive'].shift(1))
    # 为每个连续正数序列分配一个唯一的标识
    df_positive_copy['group_id'] = df_positive_copy['new_group'].cumsum()
    # 仅选取正数进行分组和计数
    positive_sequences = df_positive_copy[df_positive_copy['is_positive']].groupby('group_id').size()
    # 首先对每个连续正数序列进行聚合计算，得到波动总值、平均值和最大值
    grouped_stats = df_positive_copy[df_positive_copy['is_positive']].groupby('group_id')['Fluctuation15M'].agg(
        total_fluctuation='sum',
        average_fluctuation='mean',
        max_fluctuation='max'
    ).reset_index()

    # 将原先计算的序列长度也加入统计表中
    grouped_stats['sequence_length'] = grouped_stats['group_id'].map(positive_sequences)

    # 现在grouped_stats包含了每个连续正数序列的长度、波动总值、平均值和最大值
    # 对这些统计值进行进一步的汇总统计
    summary_stats = grouped_stats.groupby('sequence_length').agg({
        'total_fluctuation': ['sum', 'mean', 'max'],
        'average_fluctuation': ['mean'],
        'max_fluctuation': ['max']
    })
    # 计算各长度连续波动的数量
    sequence_counts = positive_sequences.groupby(positive_sequences).size().reset_index()
    sequence_counts.columns = ['sequence_length', 'count']

    # 合并各项统计数据
    summary_stats = pd.merge(summary_stats, sequence_counts, on='sequence_length', how='left')

    # 重命名列以便更清晰地表示统计结果
    summary_stats.columns = ['sequence_length', 'total_fluctuation_sum', 'total_fluctuation_mean', 'total_fluctuation_max',
                             'average_fluctuation_mean', 'max_fluctuation_max', 'count']
    # # 重命名列以便更清晰地表示统计结果
    # summary_stats.columns = ['_'.join(col).strip() for col in summary_stats.columns.values]
    # summary_stats = summary_stats.reset_index()

    return summary_stats



# 对上下波动分开统计

In [10]:
def classic_fluctuation_stata(df,folder) :# 统计典型波动优化效果
    # 函数功能：经典波动量数据
    # 函数输入：数据集、上波动分析结果路径、下波动分析结果路径
    # 函数输出：总结列表、向上向下波动统计列表（包含原上波动功率值求和、原向下波动求和功率、优化后上波动功率值求和、优化后向下波动求和功率）

    df_result = df.copy()
    # 后上下波动统计
    original_upper_amount = df_result['original_flucuation'][df_result['original_flucuation'] > upper_baseline].sum()
    original_lower_amount = df_result['original_flucuation'][df_result['original_flucuation'] < lower_baseline].sum()
    # 后上下波动统计
    after_upper_amount = df_result['after_flucuation'][df_result['after_flucuation'] > upper_baseline].sum()
    after_lower_amount = df_result['after_flucuation'][df_result['after_flucuation'] < lower_baseline].sum()

    summary_stats_list = []
    for i in ('positive','negative'):
        for j in ('original','after'):
            file_name = i+'_'+j+'_'+'summary.csv'
            stats = continuou_fluctuation_analysis(df_result,i,j)
            # 拼接文件夹路径和文件名
            file_path = os.path.join(folder, file_name)
            # 将结果保存到CSV文件
            stats.to_csv(file_path,index=False)
            summary_stats_list.append(stats)
    
    classic_fluctuation_list = [original_upper_amount,original_lower_amount,after_upper_amount,after_lower_amount]
    return summary_stats_list,classic_fluctuation_list


# 对数据绝对值处理定义

In [11]:
def absolute_data(df):
    df_abs = df.copy()
    df_abs['abs_original_flucuation'] = abs(df['original_flucuation'])
    df_abs['abs_after_flucuation'] = abs(df['after_flucuation'])
    return df_abs

data_abs_1 = absolute_data(data_anly_1)

In [12]:
# method_1_refine_num_list = refine_num(data_abs_1)
# method_1_refine_num_list
# file = 'ch5_analysis'
# data,method_1_confluctuation_list = classic_fluctuation_stata(data_abs_1,file)


In [13]:
def statistics(data,refine_folder,stata_folder):
# 功能：对波动量进行统计 计算考核电量和各重要统计量
# 输入 储能功率数据集
# 输出 
# 波动量统计数据集，
#       原连续向上波动统计表
#       原连续向下波动统计表
#       后连续向上波动统计表
#       后连续向下波动统计表
# 各重要统计量结论列表
# 如下
# 考核量列表
#   （包含减少的考核电量、
#              原考核电量、
#              后考核电量）
# 考核点统计
#              原受考核点数量、
#              原向上波动考核点数量、
#              原向下波动考核点数量、
#              优化后受考核点数量、
#              优化后向上波动考核点数量、
#              优化后向下波动考核点数量、
# 考核点比例统计
#               原受考核点比例、
#               原向上波动考核点比例、
#               原向下波动考核点比例、
#               优化后受考核点比例、
#               优化后向上波动考核点比例、
#               优化后向下波动考核点比例、
# 向上向下波动量统计
#              原向上波动考核功率求和、
#              原向上波动考核功率求和、
#              后向上波动考核功率求和、
#              后向上波动考核功率求和

    df_pre = data.copy()
    df = absolute_data(df_pre)
    df_result,refine_list = refine(df,refine_folder)# 统计考核量（需要写一段代码算出考核量）
    refine_num_list = refine_num(df)# 统计考核点的数量（）
    summary_stats_list,classic_fluctuation_list = classic_fluctuation_stata(df,stata_folder) # 统计典型波动优化效果
    
    anly_list = refine_list+refine_num_list+classic_fluctuation_list
    stata_list = [df_result,summary_stats_list]
    return stata_list,anly_list

In [14]:
print_list = ['减少的考核电量',
              '原考核电量',
              '后考核电量',
               '原受考核点数量',
               '原向上波动考核点数量',
               '原向下波动考核点数量',
               '优化后受考核点数量',
               '优化后向上波动考核点数量',
               '优化后向下波动考核点数量',
               '原受考核点比例',
               '原向上波动考核点比例',
               '原向下波动考核点比例',
               '优化后受考核点比例',
               '优化后向上波动考核点比例',
               '优化后向下波动考核点比例',
               '原向上波动考核功率求和',
               '原向上波动考核功率求和',
               '后向上波动考核功率求和',
               '后向上波动考核功率求和']


# 调用示例

## 方案1结果分析

In [15]:
# 一个标准的调用函数示例
# 输入一个数据集，

refine_foler = 'ch5_data/analyse_results_of_dates/abs'
stata_folder = 'ch5_data/analyse_results_of_dates/abs'

stata_list_1,anly_list_1 = statistics(data_anly_1,refine_foler,stata_folder)
anly_list_1

for i in range(len(print_list)):
    print(f'{print_list[i]}是{anly_list_1[i]}')



减少的考核电量是959083.0427182534
原考核电量是1107604.9910515868
后考核电量是148521.9483333334
原受考核点数量是8903
原向上波动考核点数量是4466
原向下波动考核点数量是4437
优化后受考核点数量是2439
优化后向上波动考核点数量是1214
优化后向下波动考核点数量是1225
原受考核点比例是0.2543714285714286
原向上波动考核点比例是0.1276
原向下波动考核点比例是0.12677142857142856
优化后受考核点比例是0.06968571428571428
优化后向上波动考核点比例是0.034685714285714285
优化后向下波动考核点比例是0.035
原向上波动考核功率求和是535053.67
原向上波动考核功率求和是-520099.51
后向上波动考核功率求和是154286.52000000002
后向上波动考核功率求和是-147300.3


In [16]:

visualization_interactive(data_anly_1, upper_baseline, lower_baseline,96*15,96*16)
# 200 201 非常ok
# 

## 方案2结果分析

In [17]:
# 一个标准的调用函数示例
# 输入一个数据集，

refine_foler = 'ch5_data/analyse_results_of_dates/range_width_1'
stata_folder = 'ch5_data/analyse_results_of_dates/range_width_1'

stata_list_2,anly_list_2 = statistics(data_anly_2,refine_foler,stata_folder)
for i in range(len(print_list)):
    print(f'{print_list[i]}是{anly_list_2[i]}')

减少的考核电量是656455.3279265866
原考核电量是1107604.9910515868
后考核电量是451149.6631250002
原受考核点数量是8903
原向上波动考核点数量是4466
原向下波动考核点数量是4437
优化后受考核点数量是4252
优化后向上波动考核点数量是1540
优化后向下波动考核点数量是2712
原受考核点比例是0.25408830160678103
原向上波动考核点比例是0.1274579753988413
原向下波动考核点比例是0.12663032620793974
优化后受考核点比例是0.12135049516253318
优化后向上波动考核点比例是0.04395102599960044
优化后向下波动考核点比例是0.07739946916293274
原向上波动考核功率求和是535053.67
原向上波动考核功率求和是-520099.51
后向上波动考核功率求和是159016.31000000014
后向上波动考核功率求和是-365068.17000000004


In [18]:
visualization_interactive(data_anly_2, upper_baseline, lower_baseline,96*15,96*16)


## 方案3结果分析

In [19]:
# 一个标准的调用函数示例
# 输入一个数据集，

refine_foler = 'ch5_data/analyse_results_of_dates/range_width_30'
stata_folder = 'ch5_data/analyse_results_of_dates/range_width_30'

stata_list_3,anly_list_3 = statistics(data_anly_3,refine_foler,stata_folder)
for i in range(len(print_list)):
    print(f'{print_list[i]}是{anly_list_3[i]}')

减少的考核电量是1050177.9085515868
原考核电量是1107604.9910515868
后考核电量是57427.08250000002
原受考核点数量是8903
原向上波动考核点数量是4466
原向下波动考核点数量是4437
优化后受考核点数量是2476
优化后向上波动考核点数量是1195
优化后向下波动考核点数量是1281
原受考核点比例是0.25429877177949156
原向上波动考核点比例是0.12756355327049415
原向下波动考核点比例是0.12673521850899744
优化后受考核点比例是0.07072265067123679
优化后向上波动考核点比例是0.03413310482719223
优化后向下波动考核点比例是0.03658954584404456
原向上波动考核功率求和是535053.67
原向上波动考核功率求和是-520099.51
后向上波动考核功率求和是71933.98000000016
后向上波动考核功率求和是-75182.26000000017


In [20]:
visualization_interactive(data_anly_3, upper_baseline, lower_baseline,96*15,96*16)


## 方案4结果分析

In [21]:
# 一个标准的调用函数示例
# 输入一个数据集，

refine_foler = 'ch5_data/analyse_results_of_dates/range_with_halfE'
stata_folder = 'ch5_data/analyse_results_of_dates/range_with_halfE'

stata_list_4,anly_list_4 = statistics(data_anly_4,refine_foler,stata_folder)
for i in range(len(print_list)):
    print(f'{print_list[i]}是{anly_list_4[i]}')

减少的考核电量是876987.4085515868
原考核电量是1107604.9910515868
后考核电量是230617.58250000008
原受考核点数量是8903
原向上波动考核点数量是4466
原向下波动考核点数量是4437
优化后受考核点数量是2999
优化后向上波动考核点数量是1501
优化后向下波动考核点数量是1498
原受考核点比例是0.25429877177949156
原向上波动考核点比例是0.12756355327049415
原向下波动考核点比例是0.12673521850899744
优化后受考核点比例是0.08566123964581548
优化后向上波动考核点比例是0.042873464724364464
优化后向下波动考核点比例是0.04278777492145101
原向上波动考核功率求和是535053.67
原向上波动考核功率求和是-520099.51
后向上波动考核功率求和是125955.37000000016
后向上波动考核功率求和是-120715.64000000016


In [22]:
visualization_interactive(data_anly_4, upper_baseline, lower_baseline,96*15,96*16)

In [23]:
anly_result_df = pd.DataFrame({
        '方案1':np.ceil(anly_list_1),
        '方案2': np.ceil(anly_list_2),
        '方案3': np.ceil(anly_list_3),
        '方案4':np.ceil(anly_list_4)
    })

In [24]:
anly_result_df_T = anly_result_df.T
anly_result_df_T.columns = ['减少的考核电量',
                          '原考核电量',
                          '后考核电量',
                          '原受考核点数量',
                          '原向上波动考核点数量',
                          '原向下波动考核点数量',
                          '优化后受考核点数量',
                          '优化后向上波动考核点数量',
                          '优化后向下波动考核点数量',
                          '原受考核点比例',
                          '原向上波动考核点比例',
                          '原向下波动考核点比例',
                          '优化后受考核点比例',
                          '优化后向上波动考核点比例',
                          '优化后向下波动考核点比例',
                          '原向上波动考核功率求和',
                          '原向下波动考核功率求和',
                          '后向上波动考核功率求和',
                          '后向下波动考核功率求和']

In [25]:
anly_result_df_T.T.to_csv('ch5_data/analyse_results_of_dates/各方案运行结果总体分析.csv')
anly_result_df_T['优化的电量的比例'] = np.round(anly_result_df_T['减少的考核电量']/anly_result_df_T['原考核电量'],2)
anly_result_df_T.T

,方案1,方案2,方案3,方案4
减少的考核电量,959084.00,656456.00,1050178.00,876988.00
原考核电量,1107605.00,1107605.00,1107605.00,1107605.00
后考核电量,148522.00,451150.00,57428.00,230618.00
原受考核点数量,8903.00,8903.00,8903.00,8903.00
原向上波动考核点数量,4466.00,4466.00,4466.00,4466.00
原向下波动考核点数量,4437.00,4437.00,4437.00,4437.00
优化后受考核点数量,2439.00,4252.00,2476.00,2999.00
优化后向上波动考核点数量,1214.00,1540.00,1195.00,1501.00
优化后向下波动考核点数量,1225.00,2712.00,1281.00,1498.00
原受考核点比例,1.00,1.00,1.00,1.00


# 连续波动分析

In [26]:
def continuou_fluctuation_analysis(df,mode):
    # 功能描述：分析连续波动的各项指标
    # 函数输入：原始数据集，模式，保存文件夹
    # 函数输出：分析数据集
    df_positive_copy = df.copy()

    if mode == 'positive':
        df_positive_copy['is_positive'] = df_positive_copy['after_flucuation'] > 0
        df_positive_copy['is_positive_above'] = df_positive_copy['after_flucuation'] > 50
    elif mode == 'negative':
        df_positive_copy['is_positive'] = df_positive_copy['after_flucuation'] < 0
        df_positive_copy['is_positive_above'] = df_positive_copy['after_flucuation'] < -50

    # # 进行绝对值的处理
    # df_positive_copy['is_positive'] = abs(df_positive_copy['is_positive'])
    df_positive_copy['after_flucuation'] = abs(df_positive_copy['after_flucuation'])
    
    # 检测连续正数序列的开始
    df_positive_copy['new_group'] = df_positive_copy['is_positive_above'] & (df_positive_copy['is_positive_above'] != df_positive_copy['is_positive_above'].shift(1))
    # 为每个连续正数序列分配一个唯一的标识
    df_positive_copy['group_id'] = df_positive_copy['new_group'].cumsum()
    # 仅选取正数进行分组和计数
    positive_sequences = df_positive_copy[df_positive_copy['is_positive_above']].groupby('group_id').size()
    # 首先对每个连续正数序列进行聚合计算，得到波动总值、平均值和最大值
    grouped_stats = df_positive_copy[df_positive_copy['is_positive_above']].groupby('group_id')['after_flucuation'].agg(
        total_fluctuation='sum',
        average_fluctuation='mean',
        max_fluctuation='max'
    ).reset_index()

    # 将原先计算的序列长度也加入统计表中
    grouped_stats['sequence_length'] = grouped_stats['group_id'].map(positive_sequences)

    # 现在grouped_stats包含了每个连续正数序列的长度、波动总值、平均值和最大值
    # 对这些统计值进行进一步的汇总统计
    summary_stats = grouped_stats.groupby('sequence_length').agg({
        'total_fluctuation': ['sum', 'mean', 'max'],
        'average_fluctuation': ['mean'],
        'max_fluctuation': ['max']
    })
    # 计算各长度连续波动的数量
    sequence_counts = positive_sequences.groupby(positive_sequences).size().reset_index()
    sequence_counts.columns = ['sequence_length', 'count']

    # 合并各项统计数据
    summary_stats = pd.merge(summary_stats, sequence_counts, on='sequence_length', how='left')

    # 重命名列以便更清晰地表示统计结果
    summary_stats.columns = ['sequence_length', 'total_fluctuation_sum', 'total_fluctuation_mean', 'total_fluctuation_max',
                             'average_fluctuation_mean', 'max_fluctuation_max', 'count']
    # # 重命名列以便更清晰地表示统计结果
    # summary_stats.columns = ['_'.join(col).strip() for col in summary_stats.columns.values]
    # summary_stats = summary_stats.reset_index()

    return summary_stats



In [27]:
def method_continuou_fluctuation_analysis(df):
    df_positive_copy = df.copy()
    df_negative_copy = df.copy()
    positive_summary_stats = continuou_fluctuation_analysis(df_positive_copy,'positive')
    positive_con_sum = positive_summary_stats['total_fluctuation_sum'][1:].sum() # 连续向上越限波动段功率差的和值的总和
    positive_con_max = positive_summary_stats['total_fluctuation_max'][1:].max() # 连续向上波动段和值功率差的和值的最大值
    positive_con_front_max = positive_summary_stats['total_fluctuation_max'][1:5].max() # 长度为2~5的连续向上波动段和值功率差的和值的最大值
    
    negative_summary_stats = continuou_fluctuation_analysis(df_negative_copy,'negative')
    negative_con_sum = negative_summary_stats['total_fluctuation_sum'][1:].sum() # 连续向下越限波动段功率差的和值的总和
    negative_con_max = negative_summary_stats['total_fluctuation_max'][1:].max() # 连续向下波动段和值功率差的和值的最大值
    negative_con_front_max = negative_summary_stats['total_fluctuation_max'][1:5].max() # 长度为2~5的连续向下波动段和值功率差的和值的最大值
    result_list = [positive_con_sum,positive_con_max,positive_con_front_max,negative_con_sum,negative_con_max,negative_con_front_max]

    return result_list

## 方案1

### 正向波动

In [28]:
df_positive_copy = data_anly_1.copy()
positive_summary_stats = continuou_fluctuation_analysis(df_positive_copy,'positive')
positive_summary_stats.to_csv('ch5_data/analyse_results_of_dates/方案1的连续向上越限波动统计.csv')
positive_summary_stats

,sequence_length,total_fluctuation_sum,total_fluctuation_mean,total_fluctuation_max,average_fluctuation_mean,max_fluctuation_max,count
0,1,43128.38,110.585590,832.81,110.585590,832.81,390
1,2,38076.19,247.247987,1493.01,123.623994,1274.10,154
2,3,27193.28,445.791475,1153.73,148.597158,583.94,61
3,4,20229.51,577.986000,1722.22,144.496500,670.60,35
4,5,8839.71,631.407857,1099.19,126.281571,454.23,14
5,6,8542.54,949.171111,1210.75,158.195185,642.00,9
6,7,1255.60,627.800000,636.07,89.685714,146.47,2
7,8,1220.44,1220.440000,1220.44,152.555000,244.34,1
8,9,3394.94,1131.646667,1305.12,125.738519,233.17,3
9,10,2405.93,1202.965000,1518.54,120.296500,413.28,2


### 反向波动

In [29]:
df_negative_copy = data_anly_1.copy()
negative_summary_stats = continuou_fluctuation_analysis(df_negative_copy,'negative')
negative_summary_stats.to_csv('ch5_data/analyse_results_of_dates/方案1的连续向下越限波动统计.csv')
negative_summary_stats

,sequence_length,total_fluctuation_sum,total_fluctuation_mean,total_fluctuation_max,average_fluctuation_mean,max_fluctuation_max,count
0,1,45940.59,112.324181,1177.04,112.324181,1177.04,409
1,2,35619.03,234.335724,1256.63,117.167862,740.89,152
2,3,20695.05,390.472642,944.50,130.157547,599.27,53
3,4,17407.82,543.994375,1729.90,135.998594,767.80,32
4,5,9991.65,587.744118,1117.25,117.548824,348.65,17
5,6,5257.35,751.050000,1531.35,125.175000,348.80,7
6,7,8962.93,896.293000,1355.75,128.041857,456.38,10
7,9,2393.97,1196.985000,1234.36,132.998333,248.54,2
8,10,1031.91,1031.910000,1031.91,103.191000,195.39,1


## 方案2

### 正向波动

In [30]:
df_positive_copy = data_anly_2.copy()
positive_summary_stats = continuou_fluctuation_analysis(df_positive_copy,'positive')
positive_summary_stats.to_csv('ch5_data/analyse_results_of_dates/方案2的连续向上越限波动统计.csv')
positive_summary_stats

,sequence_length,total_fluctuation_sum,total_fluctuation_mean,total_fluctuation_max,average_fluctuation_mean,max_fluctuation_max,count
0,1,60166.41,57.852317,694.68,57.852317,694.68,1040
1,2,33670.86,378.324270,1010.05,189.162135,748.95,89
2,3,23202.33,610.587632,1201.14,203.529211,735.84,38
3,4,15156.24,891.543529,1731.43,222.885882,693.17,17
4,5,13250.47,1019.266923,1777.01,203.853385,696.47,13
5,6,9601.55,1200.193750,1901.26,200.032292,821.57,8
6,8,701.80,701.800000,701.80,87.725000,129.56,1
7,9,1535.28,1535.280000,1535.28,170.586667,542.02,1
8,10,1731.37,1731.370000,1731.37,173.137000,432.64,1


In [31]:
df_negative_copy = data_anly_2.copy()
negative_summary_stats = continuou_fluctuation_analysis(df_negative_copy,'negative')
negative_summary_stats.to_csv('ch5_data/analyse_results_of_dates/方案2的连续向下越限波动统计.csv')
negative_summary_stats

,sequence_length,total_fluctuation_sum,total_fluctuation_mean,total_fluctuation_max,average_fluctuation_mean,max_fluctuation_max,count
0,1,78362.54,108.685908,738.63,108.685908,738.63,721
1,2,133228.75,285.286403,1176.71,142.643201,956.63,467
2,3,59294.84,439.221037,1866.57,146.407012,952.12,135
3,4,46911.16,642.618630,1654.71,160.654658,1001.57,73
4,5,13929.71,696.485500,1862.79,139.297100,664.04,20
5,6,10587.00,705.800000,1186.73,117.633333,555.16,15
6,7,6918.10,864.762500,1579.42,123.537500,565.21,8
7,8,7318.42,1219.736667,1902.53,152.467083,523.01,6
8,9,2235.44,1117.720000,1170.12,124.191111,269.50,2
9,11,3221.24,1610.620000,1639.97,146.420000,333.49,2


## 方案3

### 正向波动

In [32]:
df_positive_copy = data_anly_3.copy()
positive_summary_stats = continuou_fluctuation_analysis(df_positive_copy,'positive')
positive_summary_stats.to_csv('ch5_data/analyse_results_of_dates/方案3的连续向上越限波动统计.csv')
positive_summary_stats

,sequence_length,total_fluctuation_sum,total_fluctuation_mean,total_fluctuation_max,average_fluctuation_mean,max_fluctuation_max,count
0,1,47984.62,50.831165,307.06,50.831165,307.06,944
1,2,9293.30,163.040351,468.54,81.520175,349.83,57
2,3,6084.15,289.721429,839.11,96.573810,560.10,21
3,4,3545.28,590.880000,1083.91,147.720000,685.37,6
4,5,2707.46,541.492000,715.67,108.298400,225.91,5
5,6,1461.06,487.020000,699.03,81.170000,196.28,3
6,7,858.11,858.110000,858.11,122.587143,228.14,1


### 反向波动

In [33]:
df_negative_copy = data_anly_3.copy()
negative_summary_stats = continuou_fluctuation_analysis(df_negative_copy,'negative')
negative_summary_stats.to_csv('ch5_data/analyse_results_of_dates/方案3的连续向下越限波动统计.csv')
negative_summary_stats

,sequence_length,total_fluctuation_sum,total_fluctuation_mean,total_fluctuation_max,average_fluctuation_mean,max_fluctuation_max,count
0,1,54048.63,50.702280,245.32,50.702280,245.32,1066
1,2,9563.19,170.771250,559.50,85.385625,367.44,56
2,3,3427.86,380.873333,972.00,126.957778,636.34,9
3,4,2497.31,624.327500,885.43,156.081875,412.01,4
4,5,1666.95,555.650000,702.91,111.130000,211.77,3
5,6,960.81,480.405000,528.22,80.067500,144.34,2
6,8,2185.63,728.543333,924.62,91.067917,305.69,3
7,9,831.88,831.880000,831.88,92.431111,211.83,1


## 方案4

### 正向波动

In [34]:
df_positive_copy = data_anly_4.copy()
positive_summary_stats = continuou_fluctuation_analysis(df_positive_copy,'positive')
positive_summary_stats.to_csv('ch5_data/analyse_results_of_dates/方案4的连续向上越限波动统计.csv')
positive_summary_stats

,sequence_length,total_fluctuation_sum,total_fluctuation_mean,total_fluctuation_max,average_fluctuation_mean,max_fluctuation_max,count
0,1,59734.59,57.603269,724.86,57.603269,724.86,1037
1,2,21582.32,263.199024,867.98,131.599512,675.83,82
2,3,22934.45,468.050000,1393.08,156.016667,1152.93,49
3,4,11617.27,645.403889,1656.56,161.350972,708.94,18
4,5,1768.39,589.463333,928.38,117.892667,700.10,3
5,6,4336.26,867.252000,993.88,144.542000,338.05,5
6,7,1056.83,528.415000,631.87,75.487857,197.66,2
7,11,2925.26,1462.630000,1471.17,132.966364,414.28,2


### 反向波动

In [35]:
df_negative_copy = data_anly_4.copy()
negative_summary_stats = continuou_fluctuation_analysis(df_negative_copy,'negative')
negative_summary_stats.to_csv('ch5_data/analyse_results_of_dates/方案4的连续向下越限波动统计.csv')
negative_summary_stats

,sequence_length,total_fluctuation_sum,total_fluctuation_mean,total_fluctuation_max,average_fluctuation_mean,max_fluctuation_max,count
0,1,54451.04,56.309245,552.36,56.309245,552.36,967
1,2,22276.16,244.792967,1101.03,122.396484,756.66,91
2,3,15526.83,398.123846,1565.92,132.707949,1016.53,39
3,4,8189.84,481.755294,905.21,120.438824,384.25,17
4,5,4429.40,553.675000,756.83,110.735000,454.24,8
5,6,3722.38,744.476000,1211.75,124.079333,366.32,5
6,7,4756.77,951.354000,1255.31,135.907714,468.47,5
7,8,3637.57,1212.523333,1288.53,151.565417,302.97,3
8,11,2305.34,1152.670000,1229.51,104.788182,211.81,2
9,13,1420.31,1420.310000,1420.31,109.254615,204.85,1


In [36]:
def method_continuou_fluctuation_analysis(df):
    df_positive_copy = df.copy()
    df_negative_copy = df.copy()
    positive_summary_stats = continuou_fluctuation_analysis(df_positive_copy,'positive')
    positive_con_sum = positive_summary_stats['total_fluctuation_sum'][1:].sum() # 连续向上越限波动段功率差的和值的总和
    positive_con_mean = positive_summary_stats['total_fluctuation_sum'][1:].sum()/positive_summary_stats['count'][1:].sum()
    positive_con_max = positive_summary_stats['total_fluctuation_max'][1:].max() # 连续向上波动段和值功率差的和值的最大值
    positive_con_front_max = positive_summary_stats['total_fluctuation_max'][1:5].max() # 长度为2~5的连续向上波动段和值功率差的和值的最大值
    positive_count = positive_summary_stats['count'][1:].sum()
    positive_front_count = positive_summary_stats['count'][1:5].sum()

    negative_summary_stats = continuou_fluctuation_analysis(df_negative_copy,'negative')
    negative_con_sum = negative_summary_stats['total_fluctuation_sum'][1:].sum() # 连续向下越限波动段功率差的和值的总和
    negative_con_mean = negative_summary_stats['total_fluctuation_sum'][1:].sum()/positive_summary_stats['count'][1:].sum()
    negative_con_max = negative_summary_stats['total_fluctuation_max'][1:].max() # 连续向下波动段和值功率差的和值的最大值
    negative_con_front_max = negative_summary_stats['total_fluctuation_max'][1:5].max() # 长度为2~5的连续向下波动段和值功率差的和值的最大值
    negative_count = negative_summary_stats['count'][1:].sum()
    negative_front_count = negative_summary_stats['count'][1:5].sum()

    result_list = [positive_con_sum,
                   round(positive_con_mean,2),
                   positive_con_max,
                   positive_con_front_max,
                   positive_count,
                   positive_front_count,
                   negative_con_sum,
                   round(negative_con_mean,2),
                   negative_con_max,
                   negative_con_front_max,
                   negative_count,
                   negative_front_count]

    return result_list

In [37]:
# 指标统计
method_1_result_list = method_continuou_fluctuation_analysis(data_anly_1)
method_2_result_list = method_continuou_fluctuation_analysis(data_anly_2)
method_3_result_list = method_continuou_fluctuation_analysis(data_anly_3)
method_4_result_list = method_continuou_fluctuation_analysis(data_anly_4)

data_original = data_anly_1.copy()
data_original['after_flucuation'] = data_original['original_flucuation']
original_result_list = method_continuou_fluctuation_analysis(data_original)
original_result_list

[411213.3099999999,
 364.87,
 2001.29,
 1834.0900000000001,
 1127,
 1066,
 388487.55,
 344.71,
 1834.2,
 1834.2,
 1090,
 1033]

In [38]:
method_con_result_df = pd.DataFrame({
        '无储能':original_result_list,
        '方案1':method_1_result_list,
        '方案2': method_2_result_list,
        '方案3': method_3_result_list,
        '方案4':method_4_result_list
    })

In [39]:
method_con_result_df_T = method_con_result_df.T
method_con_result_df_T.columns = ['连续向上越限波动段功率差的和值的总和',
                                  '连续向上波动段功率差和值的平均值',
                                  '连续向上波动段和值功率差的和值的最大值',
                                  '长度为2~5的连续向上波动段和值功率差的和值的最大值',
                                  '连续向上波动段计数',
                                  '长度为2~5的连续向上波动段计数',
                                  '连续向下越限波动段功率差的和值的总和',
                                  '连续向下波动段功率差和值的平均值',
                                  '连续向下波动段和值功率差的和值的最大值',
                                  '长度为2~5的连续向下波动段和值功率差的和值的最大值',
                                  '连续向下波动段计数',
                                  '长度为2~5的连续向下波动段计数']
method_con_result_df_T.to_csv("ch5_data/analyse_results_of_dates/不同方案的连续越限波动统计.csv")
method_con_result_df_T

,连续向上越限波动段功率差的和值的总和,连续向上波动段功率差和值的平均值,连续向上波动段和值功率差的和值的最大值,长度为2~5的连续向上波动段和值功率差的和值的最大值,连续向上波动段计数,长度为2~5的连续向上波动段计数,连续向下越限波动段功率差的和值的总和,连续向下波动段功率差和值的平均值,连续向下波动段和值功率差的和值的最大值,长度为2~5的连续向下波动段和值功率差的和值的最大值,连续向下波动段计数,长度为2~5的连续向下波动段计数
无储能,411213.31,364.87,2001.29,1834.09,1127.0,1066.0,388487.55,344.71,1834.20,1834.20,1090.0,1033.0
方案1,111158.14,395.58,1722.22,1722.22,281.0,264.0,101359.71,360.71,1729.90,1729.90,274.0,254.0
方案2,98849.90,588.39,1901.26,1777.01,168.0,157.0,286705.63,1706.58,1902.53,1866.57,730.0,695.0
方案3,23949.36,257.52,1083.91,1083.91,93.0,89.0,21133.63,227.24,972.00,972.00,78.0,72.0
方案4,66220.78,411.31,1656.56,1656.56,161.0,152.0,66264.60,411.58,1565.92,1565.92,171.0,155.0


In [40]:
method_con_result_df_T = method_con_result_df_T.reset_index()
method_con_result_df_T

,index,连续向上越限波动段功率差的和值的总和,连续向上波动段功率差和值的平均值,连续向上波动段和值功率差的和值的最大值,长度为2~5的连续向上波动段和值功率差的和值的最大值,连续向上波动段计数,长度为2~5的连续向上波动段计数,连续向下越限波动段功率差的和值的总和,连续向下波动段功率差和值的平均值,连续向下波动段和值功率差的和值的最大值,长度为2~5的连续向下波动段和值功率差的和值的最大值,连续向下波动段计数,长度为2~5的连续向下波动段计数
0,无储能,411213.31,364.87,2001.29,1834.09,1127.0,1066.0,388487.55,344.71,1834.20,1834.20,1090.0,1033.0
1,方案1,111158.14,395.58,1722.22,1722.22,281.0,264.0,101359.71,360.71,1729.90,1729.90,274.0,254.0
2,方案2,98849.90,588.39,1901.26,1777.01,168.0,157.0,286705.63,1706.58,1902.53,1866.57,730.0,695.0
3,方案3,23949.36,257.52,1083.91,1083.91,93.0,89.0,21133.63,227.24,972.00,972.00,78.0,72.0
4,方案4,66220.78,411.31,1656.56,1656.56,161.0,152.0,66264.60,411.58,1565.92,1565.92,171.0,155.0


## 连续波动画图

In [41]:

fig = px.bar(
    x=method_con_result_df_T['index'],
    y=["连续向上越限波动段功率差的和值的总和", '连续向下越限波动段功率差的和值的总和'],
    labels={'index': '方案', 'value': '功率值 (MW)', 'variable':'分析指标'},
    data_frame=method_con_result_df_T,
    barmode='group',
)
# 更新图表的布局设置，包括字体
fig.update_layout(
    xaxis_title='方案',
    yaxis_title='功率值 (MW)',
    yaxis_title_standoff=25,
    font=dict(family="SimHei", size=20)
)




In [42]:

fig = px.bar(
    x=method_con_result_df_T['index'],
    y= ["连续向上波动段和值功率差的和值的最大值",
        '连续向下波动段和值功率差的和值的最大值',
        '长度为2~5的连续向上波动段和值功率差的和值的最大值',
        '长度为2~5的连续向下波动段和值功率差的和值的最大值',
        '连续向上波动段功率差和值的平均值',
        '连续向下波动段功率差和值的平均值'
        ],  
    data_frame=method_con_result_df_T,
    labels={'index': '方案', 'value': '功率值 (MW)', 'variable':'分析指标'},
    barmode='group',
)
fig.update_layout(
    xaxis_title='方案',
    yaxis_title='功率值 (MW)',
    yaxis_title_standoff=25,
    font=dict(family="SimHei", size=20)
)
fig.show() 

In [43]:
fig = px.bar(
    x=method_con_result_df_T['index'],
    y= ["连续向上波动段计数",
        '连续向下波动段计数',
        '长度为2~5的连续向上波动段计数',
        '长度为2~5的连续向下波动段计数'
        ],  
    data_frame=method_con_result_df_T,
    labels={'index': '方案', 'value': '段数', 'variable':'分析指标'},
    barmode='group',
)
fig.update_layout(
    xaxis_title='方案',
    yaxis_title='段数',
    yaxis_title_standoff=25,
    font=dict(family="SimHei", size=20)
)
fig.show()